In [41]:
import pandas as pd
import re
import json

import shinra_util as util

In [2]:
%load_ext autoreload
%autoreload 2

In [30]:
s = "次に酢酸鉛(II) を加えて鉛塩とし、さらに硫化水素で分解すると、遊離のメリト酸が得られる。"

In [38]:
util.get_compound_list(s)

['酢酸鉛(II)', '鉛塩', '硫化水素', '分解', '遊離', 'メリト酸']

In [31]:
extract_df = pd.DataFrame({"material": util.get_compound_list(s)})

In [27]:
noun_list = pd.read_csv("../data/noun_list_in_category_and_title.csv")
compound_list = pd.read_csv("../data/compound_list.csv")
NITE_list = pd.read_csv("../data/NITE_existence_list.csv").pipe(lambda x: x[x.label == 1])

In [34]:
compound_like_list = list(set(compound_list['compound'].tolist() + noun_list['noun'].tolist()))

In [39]:
extract_df[extract_df['material'].str.contains(util.contains_patt(compound_like_list))]

,material
0,酢酸鉛(II)
1,鉛塩
2,硫化水素
5,メリト酸


In [42]:
with open("../data/compound_train.json") as f:
    train = json.load(f)['entry']
train_dict = util.train2dict(train, attribute="原材料")

In [ ]:
def is_contains_in_tain_data():
    use_str = ','.join(true_use_list)
    result1 = group.apply(lambda x: True if re.search(x.use, use_str) else False, axis=1).tolist()
    is_contain_list1 += result1   

def is_contains_in_extraction_data(true: [list, str]):
    return df[0].str.contains(util.contains_patt(true)).tolist()
    
def validation(extraction: dict, train: dict):
    precision_list = extraction_df.extraction.str.contains(util.contains_patt(true)).tolist() \
                    +extraction_df.apply(lambda x: True if re.search(x.extraction, ','.join(true_use_list)) else False, axis=1).tolist()
    

In [ ]:
is_contain_list1 = []
is_contain_list2 = []
predict_df = pd.DataFrame()
for _id, group in use_df.groupby('_id'):
    if not train_use_dict.get(_id):
        continue
    
    group = group[~group.duplicated('use')]
    true_use_list = train_use_dict[_id]
    
    # contains extraction data in train data
    use_str = ','.join(true_use_list)
    result1 = group.apply(lambda x: True if re.search(x.use, use_str) else False, axis=1).tolist()
    is_contain_list1 += result1   

    # contains train data in extraction data
    r2 = '|'.join(true_use_list).replace('(', '\(').replace(')', '\)')
    result2 = group.use.str.contains(r2).tolist()
    is_contain_list2 += result2
    
    predict_df = predict_df.append(group.assign(label = np.array(result1) + np.array(result2)))

print(len(predict_df))
print(predict_df.label.mean())